In [ ]:
import os
import json
import time

from agents import Agent, User, Module, Orchestrator
from capabilities.skills import WebCrawler, GoogleDocsReader

In [ ]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama-3.3-70b-versatile",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
GOOGLE_CREDENTIALS_PATH = os.path.join(PROJECT_ROOT, "secrets", "arara-461102-af3f0fba23f1.json")
with open(GOOGLE_CREDENTIALS_PATH, 'r', encoding='utf-8') as json_file:
    creds = json.load(json_file)

In [ ]:
user = User(
    name="student",
    description="The human user, a student from HIAAC, who asks questions to the system."
)

meeting_agent = Agent(
    name="meeting",
    description=(
        "Answers questions *strictly* about HIAAC meetings based on the provided meeting information text. "
        "Use this agent *only* if the user's question is about HIAAC meeting details (like dates, times, agenda items, or decisions) "
        "and the relevant meeting information is supplied in the current context."
    ),
    system_message=f"""
        Follow the user's instructions based on the content extracted from the web pages.
        Today is {time.strftime("%d/%m/%Y", time.localtime())}
    """,
    skills=[
        GoogleDocsReader(
            urls=["https://docs.google.com/document/d/1pAsptw5QUqHWSx-aj47SFbHEUGDvep6q8gHhI5tVE5A/edit?usp=sharing"],
            credentials_info=creds
        )
    ],
    llm_config=llm_config,
)

website = Agent(
    name="website",
    description=(
        "Answers questions *strictly* about HIAAC researchers by fetching information from the official website. "
        "Use this agent *only* if the user's question is about HIAAC researcher information (such as research areas, publications, contact, etc.)."
    ),
    system_message="""
        Follow the user's instructions based on the content extracted from the web pages.
    """,
    skills=[
        WebCrawler(
            urls=["https://hiaac.unicamp.br/researchers/"]
        )
    ],
    llm_config=llm_config,
)

conversational = Agent(
    name="AraraBot",
    description=(
        "Conversational agent for general topics and casual chat." \
        "You are not usefull for answering questions about HIAAC meetings or researchers; other agents are specialized for that."
    ),
    system_message="""
        You are a friendly chatbot designed for casual and informal conversations.
        Do not attempt to answer questions about HIAAC meetings or researchers; other agents are specialized for that.
    """,
    llm_config=llm_config,
)

speaker_transitions = {
    user:[conversational, meeting_agent, website],
    conversational:[user],
    meeting_agent:[user],
    website:[user],
}
main_module = Module(
    name="main_module",
    agents=[user, conversational, meeting_agent, website],
    speaker_selection_method="auto",
    speaker_transitions_type="allowed",
    allowed_or_disallowed_speaker_transitions=speaker_transitions,
)

# Orchestrator to manage the conversation flow
arara_orc = Orchestrator(
    name="main_orchestrator",
    module=main_module,
    llm_config=llm_config,
)

In [ ]:
chat_result = user.talk_to(arara_orc, message="Oi tudo bem?", cache=True)

In [ ]:
from agents.helpers.graph_utils import visualize_speaker_transitions_dict
visualize_speaker_transitions_dict(speaker_transitions, [user, conversational, meeting_agent, website])